# Cartel Detection using Network Analysis

This project attempts to demonstrate the use of graph theory tools in identifying possible cartel conduct among hypothetical players in a dummy procurement dataset. Data was randomly generated. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import itertools
import matplotlib.colors as mcolors
from pyvis.network import Network
from read_data import load_data
df = load_data().set_index('DATE')

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5000 entries, 2020-01-01 to 2023-07-31
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PROJECT_ID      5000 non-null   int64  
 1   PROJECT_TYPE    5000 non-null   object 
 2   ABC             5000 non-null   float64
 3   NUM_BIDS        5000 non-null   int64  
 4   WINNER_NAME     5000 non-null   object 
 5   CONTRACT_PRICE  5000 non-null   float64
 6   DURATION        5000 non-null   int64  
dtypes: float64(2), int64(3), object(2)
memory usage: 312.5+ KB


In [3]:
df

,PROJECT_ID,PROJECT_TYPE,ABC,NUM_BIDS,WINNER_NAME,CONTRACT_PRICE,DURATION
DATE,,,,,,,
2020-01-01,3602,Infrastructure Projects,1.606239e+09,3,Denji,1.606233e+09,33
2020-01-01,3624,Goods,1.961096e+09,2,Power,1.961089e+09,11
2020-01-01,3642,Infrastructure Projects,1.980748e+08,2,Makima,1.980783e+08,13
2020-01-01,3804,Goods,1.265216e+09,1,Mikata,1.265216e+09,9
2020-01-01,3951,Consulting Services,5.962449e+08,2,Denji,5.962362e+08,4
...,...,...,...,...,...,...,...
2023-07-31,977,Infrastructure Projects,1.512355e+09,1,Reze,1.512346e+09,15
2023-07-31,2511,Goods,1.843869e+09,4,Power,1.843861e+09,12
2023-07-31,2640,Infrastructure Projects,4.920698e+08,8,Denji,4.920688e+08,18


In [4]:
# create win/loss column
df['WIN_LOSS'] = 1/df['NUM_BIDS']
df

,PROJECT_ID,PROJECT_TYPE,ABC,NUM_BIDS,WINNER_NAME,CONTRACT_PRICE,DURATION,WIN_LOSS
DATE,,,,,,,,
2020-01-01,3602,Infrastructure Projects,1.606239e+09,3,Denji,1.606233e+09,33,0.333333
2020-01-01,3624,Goods,1.961096e+09,2,Power,1.961089e+09,11,0.500000
2020-01-01,3642,Infrastructure Projects,1.980748e+08,2,Makima,1.980783e+08,13,0.500000
2020-01-01,3804,Goods,1.265216e+09,1,Mikata,1.265216e+09,9,1.000000
2020-01-01,3951,Consulting Services,5.962449e+08,2,Denji,5.962362e+08,4,0.500000
...,...,...,...,...,...,...,...,...
2023-07-31,977,Infrastructure Projects,1.512355e+09,1,Reze,1.512346e+09,15,1.000000
2023-07-31,2511,Goods,1.843869e+09,4,Power,1.843861e+09,12,0.250000
2023-07-31,2640,Infrastructure Projects,4.920698e+08,8,Denji,4.920688e+08,18,0.125000


In [5]:
# aggregate by month
df_monthly = df.groupby(
    [pd.Grouper(freq = 'M'), 'WINNER_NAME', 'PROJECT_TYPE']).agg(
        total_abc      = ('ABC', 'sum'),
        ave_abc        = ('ABC', 'mean'),
        total_bids     = ('NUM_BIDS', 'sum'),
        ave_bids       = ('NUM_BIDS', 'mean'),
        count_projects = ('PROJECT_ID','count'),
        total_cp       = ('CONTRACT_PRICE','sum'),
        ave_cp         = ('CONTRACT_PRICE','mean'),
        ave_duration   = ('DURATION','mean')
        ).reset_index()


df_monthly

,DATE,WINNER_NAME,PROJECT_TYPE,total_abc,ave_abc,total_bids,ave_bids,count_projects,total_cp,ave_cp,ave_duration
0,2020-01-31,Denji,Consulting Services,4.940888e+09,7.058411e+08,32,4.571429,7,4.940865e+09,7.058379e+08,6.857143
1,2020-01-31,Denji,Goods,3.672017e+09,7.344034e+08,30,6.000000,5,3.672030e+09,7.344059e+08,6.000000
2,2020-01-31,Denji,Infrastructure Projects,1.089068e+10,1.555811e+09,25,3.571429,7,1.089065e+10,1.555807e+09,28.857143
3,2020-01-31,Fami,Consulting Services,4.805469e+09,8.009116e+08,36,6.000000,6,4.805479e+09,8.009131e+08,8.500000
4,2020-01-31,Fami,Goods,2.138404e+09,7.128012e+08,10,3.333333,3,2.138396e+09,7.127985e+08,6.333333
...,...,...,...,...,...,...,...,...,...,...,...
1196,2023-07-31,Power,Goods,8.042308e+09,1.005289e+09,36,4.500000,8,8.042298e+09,1.005287e+09,6.750000
1197,2023-07-31,Power,Infrastructure Projects,7.474178e+08,7.474178e+08,10,10.000000,1,7.474175e+08,7.474175e+08,30.000000
1198,2023-07-31,Reze,Consulting Services,3.486021e+09,1.162007e+09,16,5.333333,3,3.486015e+09,1.162005e+09,8.666667
1199,2023-07-31,Reze,Goods,2.441413e+09,1.220707e+09,11,5.500000,2,2.441421e+09,1.220711e+09,6.000000


In [6]:
#create subsets for the dataset by project type
df_infra = df[df['PROJECT_TYPE'] == 'Infrastructure Projects']
df_goods = df[df['PROJECT_TYPE'] == 'Goods']
df_const = df[df['PROJECT_TYPE'] == 'Consulting Services']

Let's focus on Infrastructure Projects for now. Why?

In [10]:
print(df_infra['WINNER_NAME'].value_counts())
print(df_goods['WINNER_NAME'].value_counts())
print(df_const['WINNER_NAME'].value_counts())


Makima       412
Denji        392
Mikata       375
Hayakawa      86
Nayuta        65
Reze          63
Power         56
Himeno        54
Pochita       51
Fami          51
Name: WINNER_NAME, dtype: int64
Hayakawa     192
Power        183
Mikata       183
Makima       178
Pochita      177
Fami         176
Denji        170
Himeno       162
Nayuta       159
Reze         149
Name: WINNER_NAME, dtype: int64
Hayakawa     197
Power        175
Denji        173
Reze         173
Makima       169
Himeno       168
Pochita      156
Nayuta       155
Mikata       154
Fami         146
Name: WINNER_NAME, dtype: int64


Here we can see that Makima, Denji and Mikata seem to have the lion's share of contracts won for infrastructure projects, while for the other types of projects, there seems to be no issue (though we can note that Hayakawa and Power have the same market position for both types of projects)